# 第7回講義 宿題

## 課題. Tensorflowを用いて, MNISTを畳み込みニューラルネットワーク(CNN)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

**`tf` の以下のモジュールはこの回では使用できないように制限されています. 注意してください.**
```python
tf.app
tf.compat
tf.contrib
tf.erros
tf.gfile
tf.graph_util
tf.image
tf.layers
tf.logging
tf.losses
tf.metrics
tf.python_io
tf.resource_loader
tf.saved_model
tf.sdca
tf.sets
tf.summary
tf.sysconfig
tf.test
```

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [34]:
def homework(train_X, train_y, test_X):
    import time
    import matplotlib.pyplot as plt
    #%matplotlib inline
    rng = np.random.RandomState(1234)
    random_state = 42
    #Convolution layers [batchsize,picture_length,picture_width,color_channels]
    class Conv:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding)+self.b
            return self.function(u)
    class Conv_same:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='SAME'):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding)+self.b
            return self.function(u)
    #Pooling layers [batchsize,picture_length,picture_width,color_channels]
    class Pooling:
        def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME'):
            self.ksize = ksize
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    #Flatten layer    
    class Flatten:
        def f_prop(self, x):
            return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    #Full-link layers
    class Dense:
        def __init__(self, in_dim, out_dim, function=lambda x: x):
            # Xavier Initialization
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(in_dim + out_dim)),
                            high=np.sqrt(6/(in_dim + out_dim)),
                            size=(in_dim, out_dim)
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
            self.function = function

        def f_prop(self, x):
            return self.function(tf.matmul(x, self.W) + self.b)
    
    layers = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
        Conv_same((5, 5, 1, 20), tf.nn.relu),  # 28x28x 1 -> 28x28x20
        Pooling((1, 2, 2, 1)),            # 28x28x20 -> 14x14x20
        Conv((5, 5, 20, 50), tf.nn.relu), # 14x14x20 ->  10x 10x50
        Pooling((1, 2, 2, 1)),            #  10x 10x50 ->  5x 5x50
        Flatten(),
        Dense(5*5*50, 10, tf.nn.softmax)
    ]

    x = tf.placeholder(tf.float32, [None, 28, 28, 1])
    t = tf.placeholder(tf.float32, [None, 10])

    def f_props(layers, x):
        for layer in layers:
            x = layer.f_prop(x)
        return x

    y = f_props(layers, x)

    cost = -tf.reduce_mean(tf.reduce_sum(t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1)) # tf.log(0)によるnanを防ぐ
    train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    valid = tf.argmax(y, 1)
    
    n_epochs = 200
    batch_size = 100
    n_batches = train_X.shape[0]//batch_size

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    cost_plt =[]
    start_time=time.time()
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        cost_arry = []
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            _cost, _ = sess.run([cost,train], feed_dict={x: train_X[start:end], t: train_y[start:end]})
            cost_arry.append(_cost)
        cost_plt.append(np.sum(cost_arry))
        tttttt = time.time()-start_time
        print ("epoch",epoch,"time",tttttt,"cost",cost_plt[len(cost_plt)-1])

        pred_y = sess.run(valid, feed_dict={x: test_X})
        
    sess.close()
    plt.plot(np.log(cost_plt))
    
    return pred_y

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、以下のscore_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

In [35]:
rng = np.random.RandomState(1234)
filter_shape=(5, 5, 1, 20)
fan_in = np.prod(filter_shape[:3])
fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
print(fan_in)
print(fan_out)
print(rng.uniform(low=-np.sqrt(6/(fan_in + fan_out)),high=np.sqrt(6/(fan_in + fan_out)),size=filter_shape).shape)
b=np.zeros((filter_shape[3]), dtype='float32')

25
500
(5, 5, 1, 20)


# Checker Cell (for student)

In [36]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import tensorflow as tf

del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.errors,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sdca,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test
]

def load_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
    mnist_X = np.r_[mnist.train.images, mnist.test.images]
    mnist_y = np.r_[mnist.train.labels, mnist.test.labels]
    return train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))

    # validate for small dataset
    train_X_mini = train_X[:5000]
    train_y_mini = train_y[:5000]
    test_X_mini = test_X[:1000]
    test_y_mini = test_y[:1000]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(np.argmax(test_y_mini, 1), pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))
    
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

AttributeError: app

In [37]:
validate_homework()
#score_homework()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch 0 time 0.4477651119232178 cost 112.248
epoch 1 time 0.9165680408477783 cost 102.073
epoch 2 time 1.2844080924987793 cost 76.6716
epoch 3 time 1.7495510578155518 cost 44.8428
epoch 4 time 2.2337489128112793 cost 30.2637
epoch 5 time 2.7027482986450195 cost 24.2721
epoch 6 time 3.1861469745635986 cost 20.8535
epoch 7 time 3.5895297527313232 cost 18.7672
epoch 8 time 3.8957719802856445 cost 17.2489
epoch 9 time 4.18108057975769 cost 16.1913
epoch 10 time 4.464359283447266 cost 15.0552
epoch 11 time 4.749513864517212 cost 14.3718
epoch 12 time 5.0282628536224365 cost 13.4638
epoch 13 time 5.383425951004028 cost 12.8814
epoch 14 time 5.751296758651733 cost 12.3032
epoch 15 time 6.044710636138916 cost 11.8161
epoch 16 time 6.405966758728027 cost 11.3092
epoch 17 time 6.806913614273071 cost 10.98

epoch 175 time 68.2701210975647 cost 0.618849
epoch 176 time 68.57949233055115 cost 0.632125
epoch 177 time 68.9954285621643 cost 0.617897
epoch 178 time 69.39557361602783 cost 0.62319
epoch 179 time 69.81857371330261 cost 0.623622
epoch 180 time 70.14351844787598 cost 0.592882
epoch 181 time 70.56296372413635 cost 0.585933
epoch 182 time 71.04429006576538 cost 0.582841
epoch 183 time 71.58266425132751 cost 0.559291
epoch 184 time 71.96786999702454 cost 0.547082
epoch 185 time 72.26842427253723 cost 0.572429
epoch 186 time 72.61602091789246 cost 0.55382
epoch 187 time 72.95046401023865 cost 0.549432
epoch 188 time 73.3155300617218 cost 0.541283
epoch 189 time 73.66708731651306 cost 0.527619
epoch 190 time 74.00039482116699 cost 0.519624
epoch 191 time 74.49205112457275 cost 0.526253
epoch 192 time 74.9316794872284 cost 0.521971
epoch 193 time 75.37416553497314 cost 0.508803
epoch 194 time 75.81677746772766 cost 0.509743
epoch 195 time 76.10203909873962 cost 0.49514
epoch 196 time 76.40

In [38]:
score_homework()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch 0 time 4.268802881240845 cost 476.058
epoch 1 time 8.225592613220215 cost 142.745
epoch 2 time 12.664805173873901 cost 104.091
epoch 3 time 16.280956745147705 cost 83.3433
epoch 4 time 20.402664184570312 cost 70.0683
epoch 5 time 24.54227304458618 cost 60.6506
epoch 6 time 28.21463942527771 cost 54.093
epoch 7 time 31.99765110015869 cost 49.0632
epoch 8 time 36.18602919578552 cost 45.5722
epoch 9 time 40.0205717086792 cost 42.0932
epoch 10 time 43.9287805557251 cost 39.3256
epoch 11 time 48.15277671813965 cost 37.0987
epoch 12 time 52.55923390388489 cost 35.2567
epoch 13 time 56.87964963912964 cost 33.2192
epoch 14 time 60.900099992752075 cost 32.0834
epoch 15 time 64.970778465271 cost 30.7415
epoch 16 time 69.5376627445221 cost 29.3761
epoch 17 time 73.72709798812866 cost 28.5522
epoch 18

epoch 176 time 626.9168951511383 cost 2.33705
epoch 177 time 630.4615797996521 cost 2.21495
epoch 178 time 633.9781873226166 cost 2.21027
epoch 179 time 637.5183591842651 cost 2.21692
epoch 180 time 641.0449950695038 cost 2.18887
epoch 181 time 644.578147649765 cost 2.06209
epoch 182 time 648.1052837371826 cost 2.13222
epoch 183 time 651.6108617782593 cost 2.16396
epoch 184 time 655.1861293315887 cost 2.05651
epoch 185 time 658.7659091949463 cost 2.01144
epoch 186 time 662.2614605426788 cost 1.97677
epoch 187 time 665.785089969635 cost 2.01885
epoch 188 time 669.3272686004639 cost 1.98801
epoch 189 time 673.02436876297 cost 1.9141
epoch 190 time 676.7580687999725 cost 1.9175
epoch 191 time 680.4331092834473 cost 1.88799
epoch 192 time 683.9236466884613 cost 1.87841
epoch 193 time 687.4207007884979 cost 1.86805
epoch 194 time 690.9408175945282 cost 1.85462
epoch 195 time 694.4594311714172 cost 1.83836
epoch 196 time 697.9770412445068 cost 1.77749
epoch 197 time 701.5066845417023 cost 1.